In [1]:
%pip install azure-cognitiveservices-speech openai
%pip install soundfile
%pip install pyaudio
%pip install langchain-core langgraph>0.2.27
%pip install -q langchain-google-genai
%pip install -q google-generativeai
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import google.generativeai as genai
import json
import math
import requests
import sys
import time
import threading
import wave
import pyaudio

import numpy as np
import soundfile as sf

from typing_extensions import TypedDict
from typing import Any, Dict, List, Optional, Tuple

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

import yaml
import copy

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the YAML file
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Load the API key
os.environ["GOOGLE_API_KEY"] = config["google_cloud_info"]["google_cloud_api_key"]
speech_key = config["azure_info"]["azure_speech_key"]
service_region = config["azure_info"]["azure_service_region"]

In [70]:
def save_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

pron_res = None  
def pronunciation_assessment_continuous_from_file(filename):
    """Performs continuous pronunciation assessment asynchronously with input from an audio file.
        See more information at https://aka.ms/csspeech/pa"""

    import difflib
    import json
    import azure.cognitiveservices.speech as speechsdk
    global pron_res
    # Creates an instance of a speech config with specified subscription key and service region.
    # Replace with your own subscription key and service region (e.g., "westus").
    # Note: The sample is for en-US language.
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    # with open(zhcnlongtxtfilename, "r", encoding="utf-8") as t:
    #     reference_text = t.readline()
    reference_text = ""
    # Create pronunciation assessment config, set grading system, granularity and if enable miscue based on your requirement.
    enable_miscue = False
    enable_prosody_assessment = True
    pronunciation_config = speechsdk.PronunciationAssessmentConfig(
        grading_system=speechsdk.PronunciationAssessmentGradingSystem.HundredMark,
        granularity=speechsdk.PronunciationAssessmentGranularity.Phoneme,
        enable_miscue=enable_miscue)
    ## can also include topic-based content assessment if required
    if enable_prosody_assessment:
        pronunciation_config.enable_prosody_assessment()

    # Creates a speech recognizer using a file as audio input.
    language = 'en-US'
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language=language, audio_config=audio_config)
    # Apply pronunciation assessment config to speech recognizer
    pronunciation_config.apply_to(speech_recognizer)

    done = False
    recognized_words = []
    prosody_scores = []
    fluency_scores = []
    durations = []
    startOffset = 0
    endOffset = 0

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def recognized(evt: speechsdk.SpeechRecognitionEventArgs):
        global pron_res
        print("pronunciation assessment for: {}".format(evt.result.text))
        print(evt.result)
        pronunciation_result = speechsdk.PronunciationAssessmentResult(evt.result)
        print(evt.result)
        pron_res = evt.result
        # print(pronunciation_result.__dict__)
        # print("    Accuracy score: {}, prosody score: {}, pronunciation score: {}, fluency score: {}".format(
        #     pronunciation_result.accuracy_score, pronunciation_result.prosody_score, pronunciation_result.pronunciation_score,
        #     pronunciation_result.fluency_score
        # ))
        nonlocal recognized_words, prosody_scores, fluency_scores, durations, startOffset, endOffset
        recognized_words += pronunciation_result.words
        fluency_scores.append(pronunciation_result.fluency_score)
        if pronunciation_result.prosody_score is not None:
            prosody_scores.append(pronunciation_result.prosody_score)
        json_result = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult)
        jo = json.loads(json_result)
        with open("pronunciation_assessment_result.json", "w") as f:
            json.dump(jo, f, indent=4)
        nb = jo["NBest"][0]
        durations.extend([int(w["Duration"]) + 100000 for w in nb["Words"]])
        if startOffset == 0:
            startOffset = nb["Words"][0]["Offset"]
        endOffset = nb["Words"][-1]["Offset"] + nb["Words"][-1]["Duration"] + 100000

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognized)
    # (Optional) get the session ID
    speech_recognizer.session_started.connect(lambda evt: print(f"SESSION ID: {evt.session_id}"))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # Stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous pronunciation assessment
    speech_recognizer.start_continuous_recognition()
    start_time = time.time()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()
    final_words = recognized_words

    durations_sum = sum([d for w, d in zip(recognized_words, durations) if w.error_type == "None"])

    # We can calculate whole accuracy by averaging
    final_accuracy_scores = []
    for word in final_words:
        if word.error_type == 'Insertion':
            continue
        else:
            final_accuracy_scores.append(word.accuracy_score)
    accuracy_score = sum(final_accuracy_scores) / (len(final_accuracy_scores)+0.001)
    # Re-calculate the prosody score by averaging
    if len(prosody_scores) == 0:
        prosody_score = float("nan")
    else:
        prosody_score = sum(prosody_scores) / len(prosody_scores)
    # Re-calculate fluency score
    fluency_score = 0
    if startOffset > 0:
        fluency_score = durations_sum / (endOffset - startOffset) * 100
    # Calculate whole completeness score
    handled_final_words = [w.word for w in final_words if w.error_type != "Insertion"]
    # completeness_score = len([w for w in final_words if w.error_type == "None"]) / (len(handled_final_words)+0.01) * 100
    # completeness_score = completeness_score if completeness_score <= 100 else 100
    sorted_scores = sorted([accuracy_score, prosody_score, fluency_score])
    pronunciation_score = sorted_scores[0] * 0.4 + sorted_scores[1] * 0.2 + sorted_scores[2] * 0.2

    print('    Paragraph pronunciation score: {:.2f}, accuracy score: {:.2f}, prosody score: {:.2f}, fluency score: {:.2f}'.format(
        pronunciation_score, accuracy_score, prosody_score, fluency_score
    ))

    for idx, word in enumerate(final_words):
        print('    {}: word: {}\taccuracy score: {}\terror type: {};'.format(
            idx + 1, word.word, word.accuracy_score, word.error_type
        ))

In [75]:
pronunciation_assessment_continuous_from_file("input_audio_3.wav")

SESSION ID: 85713d59eebd40089f3e707fdcc60ff9
pronunciation assessment for: Yeah, I'm fine.
SpeechRecognitionResult(result_id=42d391ffd91d41d6b7501fc3325ad320, text="Yeah, I'm fine.", reason=ResultReason.RecognizedSpeech)
SpeechRecognitionResult(result_id=42d391ffd91d41d6b7501fc3325ad320, text="Yeah, I'm fine.", reason=ResultReason.RecognizedSpeech)
pronunciation assessment for: We have discontinued.
SpeechRecognitionResult(result_id=3fa43559d502450f8be20032e529b097, text="We have discontinued.", reason=ResultReason.RecognizedSpeech)
SpeechRecognitionResult(result_id=3fa43559d502450f8be20032e529b097, text="We have discontinued.", reason=ResultReason.RecognizedSpeech)
CANCELED SpeechRecognitionCanceledEventArgs(session_id=85713d59eebd40089f3e707fdcc60ff9, result=SpeechRecognitionResult(result_id=4bfaa5bc20014644af7aa4dc254d183a, text="", reason=ResultReason.Canceled))
CLOSING on SpeechRecognitionCanceledEventArgs(session_id=85713d59eebd40089f3e707fdcc60ff9, result=SpeechRecognitionResult

In [72]:
pron_res

In [73]:
pron_res.__dict__

{'_RecognitionResult__handle': <azure.cognitiveservices.speech.interop._Handle at 0x1c1b7fc7620>,
 '_offset': 10300000,
 '_duration': 18700000,
 '_result_id': '3e071c27e6ce47639183f2e09738f274',
 '_reason': <ResultReason.RecognizedSpeech: 3>,
 '_text': 'Hello, good evening. How are you?',
 '_propbag': <azure.cognitiveservices.speech.PropertyCollection at 0x1c1b848c2c0>,
 '_json': '{"Id":"3e071c27e6ce47639183f2e09738f274","RecognitionStatus":"Success","Offset":10300000,"Duration":18700000,"Channel":0,"DisplayText":"Hello, good evening. How are you?","SNR":33.92577,"NBest":[{"Confidence":0.9423816,"Lexical":"hello good evening how are you","ITN":"hello good evening how are you","MaskedITN":"hello good evening how are you","Display":"Hello, good evening. How are you?","PronunciationAssessment":{"AccuracyScore":99.0,"FluencyScore":84.0,"ProsodyScore":83.3,"CompletenessScore":100.0,"PronScore":86.6},"Words":[{"Word":"hello","Offset":10300000,"Duration":5200000,"PronunciationAssessment":{"Ac

In [61]:
pron_res.__dict__

{'_accuracy_score': 99.0,
 '_pronunciation_score': 86.6,
 '_completeness_score': 100.0,
 '_fluency_score': 84.0,
 '_prosody_score': 83.3,
 '_content_assessment_result': None,
 '_words': [<azure.cognitiveservices.speech.PronunciationAssessmentWordResult at 0x1c1b829f6b0>,
  <azure.cognitiveservices.speech.PronunciationAssessmentWordResult at 0x1c1b82d3d40>]}

In [57]:
for obj in pron_res.words:
    for obj2 in obj.phonemes:
        print(obj2.__dict__)

{'_phoneme': 'h', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'ax', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'l', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'ow', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'g', '_accuracy_score': 80.0, '_nbest_phonemes': None}
{'_phoneme': 'uw', '_accuracy_score': 74.0, '_nbest_phonemes': None}
{'_phoneme': 'd', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'iy', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'v', '_accuracy_score': 87.0, '_nbest_phonemes': None}
{'_phoneme': 'n', '_accuracy_score': 92.0, '_nbest_phonemes': None}
{'_phoneme': 'ih', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'ng', '_accuracy_score': 83.0, '_nbest_phonemes': None}
{'_phoneme': 'h', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'aw', '_accuracy_score': 100.0, '_nbest_phonemes': None}
{'_phoneme': 'ax', '_accuracy_sc

In [21]:
import azure.cognitiveservices.speech as speechsdk

In [ ]:
def pronunciation_assessment_continuous_from_file(filename):
    """Performs continuous pronunciation assessment asynchronously with input from an audio file.
        See more information at https://aka.ms/csspeech/pa"""

    import difflib
    import json
    import azure.cognitiveservices.speech as speechsdk

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    reference_text = ""
    # Create pronunciation assessment config, set grading system, granularity and if enable miscue based on your requirement.
    enable_miscue = False
    enable_prosody_assessment = True
    pronunciation_config = speechsdk.PronunciationAssessmentConfig(
        grading_system=speechsdk.PronunciationAssessmentGradingSystem.HundredMark,
        granularity=speechsdk.PronunciationAssessmentGranularity.Phoneme,
        enable_miscue=enable_miscue)
    ## can also include topic-based content assessment if required
    if enable_prosody_assessment:
        pronunciation_config.enable_prosody_assessment()

    # Creates a speech recognizer using a file as audio input.
    language = 'en-US'
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language=language, audio_config=audio_config)
    # Apply pronunciation assessment config to speech recognizer
    pronunciation_config.apply_to(speech_recognizer)

    done = False
    recognized_words = []
    prosody_scores = []
    fluency_scores = []
    durations = []
    startOffset = 0
    endOffset = 0

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def recognized(evt: speechsdk.SpeechRecognitionEventArgs):
        print("pronunciation assessment for: {}".format(evt.result.text))
        print(evt.result)
        pronunciation_result = speechsdk.PronunciationAssessmentResult(evt.result)
        # with open("pronunciation_assessment_result.json", "w") as f:
        #     json.dump(pronunciation_result.json, f, indent=4)
        print("    Accuracy score: {}, prosody score: {}, pronunciation score: {}, completeness score : {}, fluency score: {}".format(
            pronunciation_result.accuracy_score, pronunciation_result.prosody_score, pronunciation_result.pronunciation_score,
            pronunciation_result.completeness_score, pronunciation_result.fluency_score
        ))
        nonlocal recognized_words, prosody_scores, fluency_scores, durations, startOffset, endOffset
        recognized_words += pronunciation_result.words
        fluency_scores.append(pronunciation_result.fluency_score)
        if pronunciation_result.prosody_score is not None:
            prosody_scores.append(pronunciation_result.prosody_score)
        json_result = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult)
        jo = json.loads(json_result)
        nb = jo["NBest"][0]
        durations.extend([int(w["Duration"]) + 100000 for w in nb["Words"]])
        if startOffset == 0:
            startOffset = nb["Words"][0]["Offset"]
        endOffset = nb["Words"][-1]["Offset"] + nb["Words"][-1]["Duration"] + 100000

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognized)
    # (Optional) get the session ID
    speech_recognizer.session_started.connect(lambda evt: print(f"SESSION ID: {evt.session_id}"))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # Stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous pronunciation assessment
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    final_words = recognized_words

    durations_sum = sum([d for w, d in zip(recognized_words, durations) if w.error_type == "None"])

    # We can calculate whole accuracy by averaging
    final_accuracy_scores = []
    for word in final_words:
        if word.error_type == 'Insertion':
            continue
        else:
            final_accuracy_scores.append(word.accuracy_score)
    accuracy_score = sum(final_accuracy_scores) / (len(final_accuracy_scores)+0.001)
    # Re-calculate the prosody score by averaging
    if len(prosody_scores) == 0:
        prosody_score = float("nan")
    else:
        prosody_score = sum(prosody_scores) / len(prosody_scores)
    # Re-calculate fluency score
    fluency_score = 0
    if startOffset > 0:
        fluency_score = durations_sum / (endOffset - startOffset) * 100
    # Calculate whole completeness score
    handled_final_words = [w.word for w in final_words if w.error_type != "Insertion"]
    completeness_score = len([w for w in final_words if w.error_type == "None"]) / (len(handled_final_words)+0.01) * 100
    completeness_score = completeness_score if completeness_score <= 100 else 100
    sorted_scores = sorted([accuracy_score, prosody_score, completeness_score, fluency_score])
    pronunciation_score = sorted_scores[0] * 0.4 + sorted_scores[1] * 0.2 + sorted_scores[2] * 0.2 + sorted_scores[3] * 0.2

    print('    Paragraph pronunciation score: {:.2f}, accuracy score: {:.2f}, prosody score: {:.2f}, completeness score: {:.2f}, fluency score: {:.2f}'.format(
        pronunciation_score, accuracy_score, prosody_score, completeness_score, fluency_score
    ))

    for idx, word in enumerate(final_words):
        print('    {}: word: {}\taccuracy score: {}\terror type: {};'.format(
            idx + 1, word.word, word.accuracy_score, word.error_type
        ))

In [ ]:
import string
def pronunciation_assessment_continuous_from_file():
    """Performs continuous pronunciation assessment asynchronously with input from an audio file.
        See more information at https://aka.ms/csspeech/pa"""

    import difflib
    import json

    # Creates an instance of a speech config with specified subscription key and service region.
    # Replace with your own subscription key and service region (e.g., "westus").
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    # provide a WAV file as an example. Replace it with your own.
    audio_config = speechsdk.audio.AudioConfig(filename="input_audio_0.wav")

    reference_text = ""
    # create pronunciation assessment config, set grading system, granularity and if enable miscue based on your requirement.
    enable_miscue = True
    enable_prosody_assessment = True
    pronunciation_config = speechsdk.PronunciationAssessmentConfig(
        reference_text=reference_text,
        grading_system=speechsdk.PronunciationAssessmentGradingSystem.HundredMark,
        granularity=speechsdk.PronunciationAssessmentGranularity.Phoneme,
        enable_miscue=enable_miscue)
    if enable_prosody_assessment:
        pronunciation_config.enable_prosody_assessment()

    # Creates a speech recognizer using a file as audio input.
    language = 'en-US'
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language=language, audio_config=audio_config)
    # apply pronunciation assessment config to speech recognizer
    pronunciation_config.apply_to(speech_recognizer)

    done = False
    recognized_words = []
    fluency_scores = []
    prosody_scores = []
    durations = []

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def recognized(evt: speechsdk.SpeechRecognitionEventArgs):
        print('pronunciation assessment for: {}'.format(evt.result.text))
        pronunciation_result = speechsdk.PronunciationAssessmentResult(evt.result)
        print('    Accuracy score: {}, pronunciation score: {}, fluency score: {}, prosody score: {}'.format(
            pronunciation_result.accuracy_score, pronunciation_result.pronunciation_score, 
            pronunciation_result.fluency_score, pronunciation_result.prosody_score
        ))
        nonlocal recognized_words, fluency_scores, durations, prosody_scores
        recognized_words += pronunciation_result.words
        fluency_scores.append(pronunciation_result.fluency_score)
        prosody_scores.append(pronunciation_result.prosody_score)
        json_result = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult)
        jo = json.loads(json_result)
        nb = jo['NBest'][0]
        durations.append(sum([int(w['Duration']) for w in nb['Words']]))

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous pronunciation assessment
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    # we need to convert the reference text to lower case, and split to words, then remove the punctuations.

    # For continuous pronunciation assessment mode, the service won't return the words with `Insertion` or `Omission`
    # even if miscue is enabled.
    # We need to compare with the reference text after received all recognized words to get these error words.
     
    final_words = recognized_words

    # We can calculate whole accuracy by averaging
    final_accuracy_scores = []
    for word in final_words:
        if word.error_type == 'Insertion':
            continue
        else:
            final_accuracy_scores.append(word.accuracy_score)
    accuracy_score = sum(final_accuracy_scores) / (len(final_accuracy_scores)+0.001)
    # Re-calculate fluency score
    fluency_score = sum([x * y for (x, y) in zip(fluency_scores, durations)]) / (sum(durations)+0.01)
    # Calculate whole completeness score
    # Re-calculate prosody score
    prosody_score = sum(prosody_scores) / len(prosody_scores)
    pron_score = accuracy_score * 0.4 + prosody_score * 0.2 + fluency_score * 0.2 + completeness_score * 0.2

    print('    Paragraph pronunciation score: {}, accuracy score: {}, fluency score: {}, prosody score: {}'.format(
        pron_score, accuracy_score, fluency_score, prosody_score
    ))

    for idx, word in enumerate(final_words):
        print('    {}: word: {}\taccuracy score: {}\terror type: {};'.format(
            idx + 1, word.word, word.accuracy_score, word.error_type
        ))

In [27]:
pronunciation_assessment_continuous_from_file()

SESSION STARTED: SessionEventArgs(session_id=d97a51cc33c84d3f941e944e2bcabc81)
pronunciation assessment for: Hello, good evening. How are you?
    Accuracy score: 99.0, pronunciation score: 86.6, completeness score : 100.0, fluency score: 84.0, prosody score: 83.3
CANCELED SpeechRecognitionCanceledEventArgs(session_id=d97a51cc33c84d3f941e944e2bcabc81, result=SpeechRecognitionResult(result_id=bce6a9ea717147a3b481e5d345a4d9fc, text="", reason=ResultReason.Canceled))
CLOSING on SpeechRecognitionCanceledEventArgs(session_id=d97a51cc33c84d3f941e944e2bcabc81, result=SpeechRecognitionResult(result_id=bce6a9ea717147a3b481e5d345a4d9fc, text="", reason=ResultReason.Canceled))
SESSION STOPPED SessionEventArgs(session_id=d97a51cc33c84d3f941e944e2bcabc81)
CLOSING on SessionEventArgs(session_id=d97a51cc33c84d3f941e944e2bcabc81)
    Paragraph pronunciation score: 33.4599999905618, accuracy score: 0.0, completeness score: 0.0, fluency score: 83.99999995280898, prosody score: 83.3
    1: word: hello	ac